인간 피드백을 인간 검토 워크플로(Human Review Workflows)에 통합하기 위해서는 다음 자원이 필요합니다.

* 데이터 세트에 레이블을 지정하기 위한 **작업자(Workforce)**: 아마존 메커니컬 터크(Amazon Mechanical Turk; MTurk) 인력, 벤더 관리 인력 또는 자체 비공개 인력을 만들어 인간 검토를 수행할 수 있습니다. 어떤 유형의 인력을 선택하든 아마존 그라운드 트루스(Amazon SageMaker Ground Truth)는 작업자에게 작업을 보내는 것을 처리합니다.


* 작업자용 UI(Human Task UI)를 생성하기 위한 **작업자 테스크 템플릿(Worker Task Template)**. 작업자 UI는 문서나 이미지와 같은 입력 데이터를 표시하고, 작업자에게 제공되는 지침을 보여줍니다. 또한, 작업자가 작업을 완료하는 데 사용할 수 있는 대화형 도구를 제공합니다. 자세한 내용은 여기(https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-instructions-overview.html)에서 확인할 수 있습니다.



* 인간 검토 워크플로를 생성하기 위한 **흐름 정의(Flow Definition)**. 흐름 정의를 사용해 인력을 구성하고, 인간 검토 작업을 수행하는 방법에 대한 정보를 제공합니다. 세이지메이커 그라운드 트루스 콘솔 또는 API를 통해 흐름 정의를 생성할 수 있습니다. 이 두 가지 옵션에 대한 자세한 내용은 여기(https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html)에서 확인할 수 있습니다.


* **휴먼 루프(Human Loop)**는 인간 검토 워크플로를 시작합니다. 내장 작업 유형 중 하나를 사용할 때 흐름 정의에 지정된 조건이 충족되거나 조건이 지정되지 않은 경우 각 객체에 대해 해당 AWS 서비스가 사용자를 대신해 휴먼 루프를 생성하고 시작합니다. 휴먼 루프가 트리거되면 흐름 정의에 지정된 대로 작업자에게 인간 검토 작업이 전송됩니다.


이 노트북에 표시된 것처럼 사용자 지정 작업 유형을 사용할 때는 AWS API를 사용해 휴먼 루프를 시작합니다. 사용자 지정 애플리케이션에서 StartHumanLoop를 호출하면 검토자에게 작업이 전송됩니다.

In [1]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=33163763712, available=31066292224, percent=6.3, used=1672310784, free=28696199168, active=478732288, inactive=3558096896, buffers=4304896, cached=2790948864, shared=638976, slab=181903360)


In [2]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
import io
import json
import uuid
import time
import boto3
import botocore

# 아마존 파이썬 SDK 클라이언트
sagemaker = boto3.client("sagemaker", region)
a2i = boto3.client("sagemaker-a2i-runtime")
s3 = boto3.client("s3", region)

# 워크플로 결과를 위해 S3 출력 위치 설정

In [4]:
output_path = f"s3://{bucket}/ground-truth-star-rating-results"
print(output_path)

s3://sagemaker-us-east-1-843956416240/ground-truth-star-rating-results


# 인력과 작업팀 설정

인력은 데이터 세트에 레이블을 지정하도록 선택한 작업자 그룹입니다. 비공개 인력을 사용할 때는 작업팀도 만드는데, 이는 아마존 오그먼티드 AI(Amazon Augmented AI; Amazon A2I) 인간 검토 작업에 할당된 인력 중 작업자 그룹입니다. 여러 작업팀을 보유할 수 있으며 각 작업에 하나 이상의 작업팀을 할당할 수 있습니다.

새로운 인력과 작업팀을 만들려면 다음으로 이동하세요.


In [5]:
print(
    "https://{}.console.aws.amazon.com/sagemaker/groundtruth?region={}#/labeling-workforces/create".format(
        region, region
    )
)

https://us-east-1.console.aws.amazon.com/sagemaker/groundtruth?region=us-east-1#/labeling-workforces/create


<img src="img/augmented-create-workforce.png" width="80%" align="left">

# 이메일 확인 - 스팸 메일함도 확인하세요

<img src="img/augmented-create-workforce-confirmation-email.png" width="60%" align="left">

# 인력 상태 검토

<img src="img/augmented-create-workforce-confirmation.png" width="80%" align="left">

# `workteam_arn` 설정

In [6]:
import boto3

account_id = boto3.client("sts").get_caller_identity().get("Account")

augmented_ai_workteam_arn = "arn:aws:sagemaker:{}:{}:workteam/private-crowd/dsoaws".format(region, account_id)

print(augmented_ai_workteam_arn)

arn:aws:sagemaker:us-east-1:843956416240:workteam/private-crowd/dsoaws


역할에 필요한 권한을 추가하려면 다음 링크 방문하세요. https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-permissions-security.html

# 작업자 작업 템플릿을 사용해 인간 작업 UI 생성

UI 템플릿을 제공해 인간 작업 UI 리소스를 만듭니다. 이 템플릿은 인간의 개입이 필요할 때마다 작업자에게 표시됩니다.

아래에서는 AWS Comprehend의 감정 분석 API와 호환되는 간단한 데모 템플릿을 제공했습니다.

다른 사전 구축된 UI(70개 이상)는 https://github.com/aws-samples/amazon-a2i-sample-task-uis 를 확인하세요.

# 주어진 프롬프트에 대해 LLM(이전 단계에서 배포됨)에서 생성된 star_rating의 순위를 매기도록 작업자에게 요청하기

이전 단계에서 LLM이 예측한 star_rating 집합을 기반으로 주어진 프롬프트에 대한 star_rating의 순위를 매기도록 작업자에게 요청할 것입니다. 이러한 순위는 향후 단계에서 보상 모델을 학습하는 데 사용됩니다.

In [7]:
template = r"""
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
  <short-instructions>
      Select the correct ranking from the dropdown (High, Low).
  </short-instructions>

  <full-instructions header="Ranking Instructions">
      Select the correct ranking from the dropdown (High, Low).
  </full-instructions>

  <p>
      For the following question <br/><b>{{ task.input.taskObject.prompt }}</b><br/> select the ranking (Low, High) for the following answer <br/><b>{{ task.input.taskObject.responses[0] }}</b></br>
  </p>
  <div>
      <select name="response_1_ranking" required>
          <option disabled selected value> -- select an option -- </option>
          <option value="1">High</option>
          <option value="2">Low</option>
      </select>
  </div>
  <p>
      For the following question <br/><b>{{ task.input.taskObject.prompt }}</b><br/> select the ranking (Low, High) for the following answer <br/><b>{{ task.input.taskObject.responses[1] }}</b></br>
  </p>
  <div>
      <select name="response_2_ranking" required>
          <option disabled selected value> -- select an option -- </option>
          <option value="1">High</option>
          <option value="2">Low</option>
      </select>
  </div>
</crowd-form>
"""

In [8]:
# 작업 UI 이름 - 이 값은 계정 및 지역별로 고유합니다. 여기에 직접 값을 입력할 수도 있습니다.
task_ui_name = "ui-ranking-" + str(uuid.uuid4())

# 작업자용 UI 리소스를 생성하세요.
human_task_ui_response = sagemaker.create_human_task_ui(HumanTaskUiName=task_ui_name, UiTemplate={"Content": template})
human_task_ui_arn = human_task_ui_response["HumanTaskUiArn"]
print(human_task_ui_arn)

arn:aws:sagemaker:us-east-1:843956416240:human-task-ui/ui-ranking-2284fe14-6f9d-4d9b-9203-5d71f0d53415


# 흐름 정의 생성하기

이 섹션에서는 흐름 정의(Flow Definition)를 생성할 것입니다. 흐름 정의를 통해 다음을 지정할 수 있습니다.

* 작업이 전송될 인력.
* 인력이 받게 될 지침. 이를 작업자 작업 템플릿이라고 합니다.
* 작업을 받는 작업자 수와 작업 완료 시간 제한을 포함한 작업자 작업의 구성.
* 출력 데이터가 저장될 위치.

이 데모는 API를 사용할 것이지만, 선택적으로 콘솔에서도 이 워크플로 정의를 생성할 수 있습니다.

자세한 내용과 지침은 https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html 을 참고하세요.

In [9]:
import uuid

# 흐름 정의(Flow definition)명 - 이 값은 계정 및 지역별로 고유합니다. 여기에 직접 값을 입력할 수도 있습니다.
flow_definition_name = "fd-ranking-" + str(uuid.uuid4())

create_workflow_definition_response = sagemaker.create_flow_definition(
    FlowDefinitionName=flow_definition_name,
    RoleArn=role,
    HumanLoopConfig={
        "WorkteamArn": augmented_ai_workteam_arn,
        "HumanTaskUiArn": human_task_ui_arn,
        "TaskCount": 1,
        "TaskDescription": "Rank the answer for the given question from Low (Worst) to High (Best)",
        "TaskTitle": "Rank the answer for the given question from Low (Worst) to High (Best)",

    },
    OutputConfig={"S3OutputPath": output_path},
)

augmented_ai_flow_definition_arn = create_workflow_definition_response["FlowDefinitionArn"]

# 위에 ^^^^ 오류가 표시되면, 먼저 개인 인력을 생성해야 합니다. 위의 단계를 참조하세요. 그런 다음 이 셀을 다시 실행하세요.

In [10]:
# 흐름 정의 설명 - 상태가 "활성화됨"으로 변경되어야 합니다.
for x in range(60):
    describeFlowDefinitionResponse = sagemaker.describe_flow_definition(FlowDefinitionName=flow_definition_name)
    print(describeFlowDefinitionResponse["FlowDefinitionStatus"])
    if describeFlowDefinitionResponse["FlowDefinitionStatus"] == "Active":
        print("Flow Definition is active")
        break
    time.sleep(2)

Active
Flow Definition is active


In [11]:
%store augmented_ai_flow_definition_arn

Stored 'augmented_ai_flow_definition_arn' (str)


In [12]:
%store augmented_ai_workteam_arn

Stored 'augmented_ai_workteam_arn' (str)


# 리소스 해제하기

In [13]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>

<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}
</script>